In [18]:
from datetime import datetime

from pymongo import MongoClient, InsertOne, UpdateOne
import mlflow


from omegaconf import OmegaConf
import pandas as pd
# import urllib.request
# import tempfile
# from gcsfs import GCSFileSystem



In [2]:

def mongo_client(config_file_path):
    """Funcion to turn mongo client ON"""
    config = OmegaConf.load(config_file_path)
    server_ip = config.main.server_ip
    port = config.main.port
    if config.main.login:
        user = config.main.user
        passwd = config.main.passwd
        client = MongoClient(f'mongodb://{server_ip}:{port}',
                            username=user,
                            password=passwd)
    else:
        client = MongoClient(f'mongodb://{server_ip}:{port}')

    return client



class FileHandler():
    """Class to get file and save it to GCP."""
    def __init__(self, gc_project):
        self.file_system = GCSFileSystem(project=gc_project)

    def spot_download_helper(self, url, dest_name):
        """Function to get file and save it to GCP."""
        dest_file = 'tv_spots_store/'+dest_name
        print(f"FILENAME:  {dest_file}")
        self.file_system.touch(dest_file)
        with self.file_system.open(dest_file, mode='wb') as file:
            with tempfile.NamedTemporaryFile() as bin_file:
                try:
                    urllib.request.urlretrieve(url, bin_file.name)
                    data = bin_file.read()
                    file.write(data)
                except Exception as e:
                    return str(e)
            file_url = file.info()
        return (dest_file, file_url)

In [5]:
# START SERVER IF NEEDED - docker run --name testing-mongo -p 27017:27017 mongo:latest
server = mongo_client('conf/mongo-config.yaml')
db = server.TV_scan


In [6]:
#Is it necessary to update?


mlflow.run('components/spotreg_update')


2023/02/08 10:51:31 INFO mlflow.projects.docker: === Building docker image register_update:f27ab8b ===
2023/02/08 10:51:32 INFO mlflow.projects.utils: === Created directory /tmp/tmp72z5pcbu for downloading remote URIs passed to arguments of type 'path' ===
2023/02/08 10:51:32 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/laertio/projects/dev/tv_scan/mlruns:/mlflow/tmp/mlruns -v /home/laertio/projects/dev/tv_scan/mlruns/0/52410d5254fe4292b60115cbb6d513d9/artifacts:/home/laertio/projects/dev/tv_scan/mlruns/0/52410d5254fe4292b60115cbb6d513d9/artifacts -v /home/laertio/projects/dev/tv_scan/components/spotreg_update/data_in:/data_in -v /home/laertio/projects/dev/tv_scan/conf:/conf -e MLFLOW_RUN_ID=52410d5254fe4292b60115cbb6d513d9 -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 register_update:9513e08 python run.py tv_scan_test input_sample.xlsx /conf/mongo-config.yaml spots_inventory --wandblogin=ea7d4ed782356f66850586612f115565

In [ ]:
#Bellow the name of the collections
#db['spots_inventory']
#db['Proc_Jobs']

In [52]:
# https://www.programmingfunda.com/filter-records-in-mongodb-using-python/
# simple query - {'field1':'value', 'field2':'value'}
# number comparison - {"field": {"$gt": 25}} OR {"field": {"$lt": 25}} OR {"field": {"$lt": 25, "$gt": 25}}
# in list of values {'field':{$in:['item1', 'item2']}}
# Regex - {"field": {"$regex": "^M"}}
################
# MORE OPERATORS - https://www.mongodb.com/docs/manual/reference/operator/query/
################
filtering = {'SpotCode':417315}

In [53]:
cursor = db['spots_inventory'].find(filtering)

In [54]:
#INSPECT CURSOR
docs_toinspect = []
for doc in cursor[1:5]:
    # Building parameters
    docs_toinspect.append(doc)



In [55]:
docs_toinspect

[{'_id': ObjectId('63e37ec23265fd3b1ffcc9bb'),
  'SpotCode': 417315,
  'SpotDesc': 'LIDER DE VENDAS PT 2021.MELHOR MARCA AUTO 9\\ANO-SH SICNOT',
  'VehicleDesc': 'SIC Noticias',
  'BrandDesc': 'PEUGEOT',
  'SubBrandDesc': nan,
  'SectorDesc': 'INDUSTRIA AUTOMOVEL',
  'CategoryDesc': 'VEICULOS A MOTOR',
  'ClassDesc': 'AUTOMOVEIS',
  'SubClassDesc': 'OUTROS OU DIVERSOS',
  'AdvertiserDesc': 'PEUGEOT PORTUGAL AUTOMOVEIS SA',
  'MediaFile': 417315,
  'MediaFileOldUrl': 'http://e-sol.mediamonitor.pt/SOLFILES/TELEVISAO/20220103/417315.wmv',
  'GCP_path': 'tv_spots_store/417315_PEUGEOT.wmv',
  'spot_store_filename': 'https://www.googleapis.com/storage/v1/b/tv_spots_store/o/417315_PEUGEOT.wmv',
  'audtrans_lang': 'pt',
  'audtrans_tiny': ' O momento é de quem o acar, com o caro. E este é o momento de assumirmos a nossa natureza. Somos liões, Somos liadas. Apesou a lidar em Portugal. Foi a marca mais vendida em 2021, e pelo no ano com o que vivo, ele é o que a melhor marca automóvel na escolha

In [56]:
cursor = db['spots_inventory'].find(filtering)

docs_toproc = []
for doc in cursor:
    # Building parameters
    doc_id = doc['_id']
    docs_toproc.append(doc_id)
    

In [57]:
job_reference = 'test_medium_peugeot'
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")



db['Proc_Jobs'].insert_one({'doc_list': docs_toproc, 'j_ref':job_reference, 'status':'Pending Processing.', 'ins_timestamp':now})

In [14]:
# GET DATA


run_parameters = {'job_reference':job_reference}



mlflow.run('components/get_data', parameters=run_parameters)

2023/02/08 11:12:07 INFO mlflow.projects.docker: === Building docker image get_data:f27ab8b ===
2023/02/08 11:12:07 INFO mlflow.projects.utils: === Created directory /tmp/tmpar5wfpw3 for downloading remote URIs passed to arguments of type 'path' ===
2023/02/08 11:12:07 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/laertio/projects/dev/tv_scan/mlruns:/mlflow/tmp/mlruns -v /home/laertio/projects/dev/tv_scan/mlruns/0/fae19725e9bf49a9accba87b96fcc669/artifacts:/home/laertio/projects/dev/tv_scan/mlruns/0/fae19725e9bf49a9accba87b96fcc669/artifacts -v /home/laertio/projects/dev/tv_scan/conf:/conf -e MLFLOW_RUN_ID=fae19725e9bf49a9accba87b96fcc669 -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 get_data:9513e08 python run.py tv_scan_test spots_inventory /conf/mongo-config.yaml test_peugeot --wandblogin ea7d4ed782356f66850586612f115565b6a8a0c3' in run with ID 'fae19725e9bf49a9accba87b96fcc669' === 
wandb: W&B API key is configured. U

In [58]:
# FEATURE EXTRACTION


run_parameters = {'job_reference':job_reference, 'model_type':'medium'}

mlflow.run('components/feature_extraction', parameters=run_parameters)


2023/02/08 15:46:54 INFO mlflow.projects.docker: === Building docker image feature_extraction:f27ab8b ===
2023/02/08 15:46:54 INFO mlflow.projects.utils: === Created directory /tmp/tmp7wgft3f9 for downloading remote URIs passed to arguments of type 'path' ===
2023/02/08 15:46:54 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/laertio/projects/dev/tv_scan/mlruns:/mlflow/tmp/mlruns -v /home/laertio/projects/dev/tv_scan/mlruns/0/210c0592ff1f46c4b0124b2ad15b7ab7/artifacts:/home/laertio/projects/dev/tv_scan/mlruns/0/210c0592ff1f46c4b0124b2ad15b7ab7/artifacts -v /home/laertio/projects/dev/tv_scan/conf:/conf -e MLFLOW_RUN_ID=210c0592ff1f46c4b0124b2ad15b7ab7 -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 feature_extraction:9513e08 python run.py tv_scan_test spots_inventory /conf/mongo-config.yaml test_medium_peugeot medium --wandblogin ea7d4ed782356f66850586612f115565b6a8a0c3' in run with ID '210c0592ff1f46c4b0124b2ad15b7ab7' === 
w

In [59]:
# CHECK - after feature extraction
job = db['Proc_Jobs'].find_one({'j_ref':job_reference})

cursor = db['spots_inventory'].find({'_id':{'$in':job['doc_list']}})

records = []
for doc in cursor:
    records.append(doc)

In [60]:
df = pd.DataFrame.from_records(records)

In [61]:
df.head()

,_id,SpotCode,SpotDesc,VehicleDesc,BrandDesc,SubBrandDesc,SectorDesc,CategoryDesc,ClassDesc,SubClassDesc,AdvertiserDesc,MediaFile,MediaFileOldUrl,GCP_path,spot_store_filename,audtrans_lang,audtrans_tiny,audtrans_small,audtrans_medium
0,63e37ec23265fd3b1ffcc9bb,417315,LIDER DE VENDAS PT 2021.MELHOR MARCA AUTO 9\AN...,SIC Noticias,PEUGEOT,NaN,INDUSTRIA AUTOMOVEL,VEICULOS A MOTOR,AUTOMOVEIS,OUTROS OU DIVERSOS,PEUGEOT PORTUGAL AUTOMOVEIS SA,417315,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...,tv_spots_store/417315_PEUGEOT.wmv,https://www.googleapis.com/storage/v1/b/tv_spo...,pt,"O momento é de quem o acar, com o caro. E est...",O momento é de quem o agarra com o carro. E e...,"O momento é de quem o agarra, com um carro. E..."


In [62]:
df.loc[0,'audtrans_tiny']

' O momento é de quem o acar, com o caro. E este é o momento de assumirmos a nossa natureza. Somos liões, Somos liadas. Apesou a lidar em Portugal. Foi a marca mais vendida em 2021, e pelo no ano com o que vivo, ele é o que a melhor marca automóvel na escolha do consumidor. Por clíderas, é natureza dos liões.'

In [63]:
df.loc[0,'audtrans_small']

' O momento é de quem o agarra com o carro. E este é o momento de assumirmos a nossa natureza. Somos liões, somos líderes. Aparjou-a líder em Portugal. Foi a marca mais vendida em 2021 e pelo 9º ano consecutivo, eleita a melhor marca automóvel na escolha do consumidor. Porque liderar é a natureza dos liões.'

In [64]:
df.loc[0,'audtrans_medium']

' O momento é de quem o agarra, com um carro. E este é o momento de assumirmos a nossa natureza. Somos leões. Somos líderes. A Pajou é líder em Portugal. Foi a marca mais vendida em 2021 e pelo 9º ano consecutivo eleita a melhor marca automóvel na escolha do consumidor. Porque liderar é a natureza dos leões.'